# Классификация методом RandomForest

Этот пример призван проиллюстрировать возможности классификации методом RandomForest и оценки значимости критериев.
https://github.com/aikula/DataDriven/blob/master/RandomForest.ipynb


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df=pd.read_excel('C:\wb_teapot.xlsx', sheets='Sheet1') #!!!!!! измените имя файла и название рабочего листа
#df = pd.read_excel('https://github.com/aikula/DataDriven/blob/2b36dbf6c74137ea845690d77324c50200f347cd/wb_teapot.xlsx') #!!!!!! измените имя файла и название рабочего листа
#df = pd.read_excel('https://docs.google.com/spreadsheets/d/1X2e0qAi9CDk4gyhqGbYO9asaiNZo-e_-/edit?usp=sharing&ouid=113103704293370634751&rtpof=true&sd=true') #!!!!!! измените имя файла и название рабочего листа


#df = pd.read_csv('https://raw.githubusercontent.com/jorisvandenbossche/pandas-tutorial/master/data/titanic.csv')


df.sample(5) #эта команда выведет пять случайных строк таблицы, таблица не отобразиться полностью.

TypeError: ignored

In [ ]:
df.info() #общая информация о столбцах, типах и пропущенных значениях

In [ ]:
df.describe() #общие статистики

In [ ]:
df['Review'].hist(bins=30, figsize=(20,4)); # как распределено количество отзывов

In [ ]:
df[df['Review']>10]['sale_june'].count()

Попробуем предсказать будет ли у модели больше 10 отзывов или нет. В этмо случае у нас получается сбалансированный набор данных. Ниже выведем список столбцов и отберем те, которые будем использовать для классификатора.

In [ ]:
df.columns # список столбцов нашего датасета 

In [ ]:
col=['Star', 'Value', 'brandId', 'ordersCount', 
       'qualityRate', 'Вес с упаковкой (кг)', 'Длина кабеля',
       'Количество температурных режимов', 'Материал корпуса',
       'Мощность устройства', 'Объем чайника', 'Страна бренда',
       'Страна производитель', 'Цвет'] # !!!!! Укажите здесь названия столбцов своего датасета!

# код ниже преобразует категорийные данные в переменные и заполняет пропуски наиболее вероятным значением
X=pd.DataFrame()
for i in col:
    if df[i].dtype.name != 'object':
        X[i]=df[i].copy()
        X.loc[X[i].isna(), i]=X[i].median()
    else:
        X[i]=pd.factorize(df[i])[0]

In [ ]:
# результат, подготовленные данные
X.sample(3)

In [ ]:
# Y будет равен нулю если отзывов 10 и меньше, и единице если больше 10
Y=df['Review'].apply(lambda x: 1 if x>10 else 0).values

In [ ]:
#разделим набор на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# создаем и тренируем модель, отдельно можно провести подбор параметров для повышения точности
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
print("Значимость признака:")
ar_f.sort(reverse=True)
ar_f

In [ ]:
#удобнее отобразить на столбчатой диаграмме
d_first = len(col)
plt.figure(figsize=(8, 8))
plt.title("Значимость признака")
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(col)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first]);

In [ ]:
# как выглядит результат предсказания для тестовой выборки
model.predict(X_test)

In [ ]:
# как выглядядт результаты тестового набора
y_test

In [ ]:
# метрика r2
r2_score(model.predict(X_test), y_test)

In [ ]:
from sklearn import metrics
# метрика, насколько точно мы предсказываем правильные значения как для 0, так и 1
print("Accuracy:",metrics.accuracy_score(y_test, model.predict(X_test)))

In [ ]:
# матрица количества правильно и ошибочно угаданных классов
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

In [ ]:
# так же матрица в процентах и более изящном виде
matrix = confusion_matrix(y_test, model.predict(X_test))
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['<10', '>10']                 # !!!!!! указать названия классов!
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Предсказанные классы')
plt.ylabel('Истинные классы')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

In [ ]:
# Еще базовые метрики оценки точности модели
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))